In [31]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask
import warnings
import xarray as xr
import glob
import os
import datetime

fpath1 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/Precipitation Analysis/precip_components_past'
regions = {'RGI01':'alaska', 'RGI02':'westerncanada', 'RGI06':'iceland', 'RGI08':'scandinavia', 
           'RGI10':['East', 'North', 'Altay', 'Chukotka'], 'RGI11':'centraleurope', 'RGI12':'caucasus', 'RGI13':'centralasiaN', 
           'RGI14': 'centralasiaW', 'RGI15':'centralasiaS', 'RGI16':'Andes', 'RGI17':'southernandes',
           'RGI18':'newzealand'}

fpath2 = 'PAST/'

In [32]:
basins = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202', 'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 'BAKER':'3404',
            'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232', 'COLORADO':'3212', 
            'CHICO':'3209', 'TORNEALVEN':'6255', 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 
            'KUBAN':'6223', 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
            'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101', 'CLUTHA':'5406', 'YUKON':'4435', 'TAKU':'4431', 
             'SUSITNA':'4430','STIKINE':'4428', 'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
            'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408', 'COLUMBIA':'4406', 'ALSEK':'4401', 'NELSON':'4125', 
              'MACKENZIE':'4123','COLVILLE':'4110', 'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 
              'TALAS':'2913', 'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 
              'YELLOW RIVER':'2434', 'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 
              'INDUS':'2309', 'GANGES':'2306','BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103','YANGTZE' : '2433'}

#### Loading in solid precip data

In [33]:
regional_solid_prcp = {}

for RGI_region, region_name in regions.items():
    if isinstance(region_name, list):  # Check if region_name is a list
        combined_data = None
        for subregion in region_name:
            file_path = os.path.join(fpath1, RGI_region, fpath2, f"{subregion}_Accumulation_sfc_r1.dat")
            temp_df = pd.read_csv(file_path, sep='\s+', header=None, skiprows=1, index_col=0)
            temp_df.index = temp_df.index.map(lambda x: str(x).zfill(5))
            temp_df.index = RGI_region[3:5] + '.' + temp_df.index.astype(str)
            if combined_data is None:
                combined_data = temp_df
            else:
                combined_data = combined_data.add(temp_df, fill_value=0)  # Add with proper alignment and fill missing values with 0
        regional_solid_prcp[RGI_region] = combined_data
    else:
        file_path = os.path.join(fpath1, RGI_region, fpath2, f"{region_name}_Accumulation_sfc_r1.dat")
        temp_df = pd.read_csv(file_path, sep='\s+', header=None, skiprows=1, index_col=0)
        temp_df.index = temp_df.index.map(lambda x: str(x).zfill(5))
        temp_df.index = RGI_region[3:5] + '.' + temp_df.index.astype(str)
        regional_solid_prcp[RGI_region] = temp_df

#### Loading in liquid precip data

In [35]:
regional_liq_prcp = {}

for RGI_region, region_name in regions.items():
    if isinstance(region_name, list):  # Check if region_name is a list
        combined_data = None
        for subregion in region_name:
            file_path = os.path.join(fpath1, RGI_region, fpath2, f"{subregion}_Rain_sfc_r1.dat")
            temp_df = pd.read_csv(file_path, sep='\s+', header=None, skiprows=1, index_col=0)
            temp_df.index = temp_df.index.map(lambda x: str(x).zfill(5))
            temp_df.index = RGI_region[3:5] + '.' + temp_df.index.astype(str)
            if combined_data is None:
                combined_data = temp_df
            else:
                combined_data = combined_data.add(temp_df, fill_value=0)  # Add with proper alignment and fill missing values with 0
        regional_liq_prcp[RGI_region] = combined_data
    else:
        file_path = os.path.join(fpath1, RGI_region, fpath2, f"{region_name}_Rain_sfc_r1.dat")
        temp_df = pd.read_csv(file_path, sep='\s+', header=None, skiprows=1, index_col=0)
        temp_df.index = temp_df.index.map(lambda x: str(x).zfill(5))
        temp_df.index = RGI_region[3:5] + '.' + temp_df.index.astype(str)
        regional_liq_prcp[RGI_region] = temp_df

In [39]:
# Importing area data
areas = {}

for RGI_region, region_name in regions.items():
    if isinstance(region_name, list):  # Check if region_name is a list
        combined_data = None
        for subregion in region_name:
            file_path = os.path.join(fpath1, RGI_region, fpath2, f"{subregion}_Area_r1.dat")
            temp_df = pd.read_csv(file_path, sep='\s+', index_col="ID")
            temp_df.index = temp_df.index.map(lambda x: str(x).zfill(5))
            temp_df.index = RGI_region[3:5] + '.' + temp_df.index.astype(str)
            temp_df = temp_df.mul(1e6)  # Convert km^2 to m^2
            if combined_data is None:
                combined_data = temp_df
            else:
                combined_data = combined_data.add(temp_df, fill_value=0)  # Add with proper alignment and fill missing values with 0
        areas[RGI_region] = combined_data
    else:
        file_path = os.path.join(fpath1, RGI_region, fpath2, f"{region_name}_Area_r1.dat")
        temp_df = pd.read_csv(file_path, sep='\s+', index_col="ID")
        temp_df.index = temp_df.index.map(lambda x: str(x).zfill(5))
        temp_df.index = RGI_region[3:5] + '.' + temp_df.index.astype(str)
        areas[RGI_region] = temp_df.mul(1e6)  # Convert km^2 to m^2

In [41]:
# Create new index using pandas date_range function
start_date = datetime.date(1980, 1, 1)
end_date = datetime.date(2020, 1, 1)
new_indices = pd.date_range(start_date, end_date, freq='A').strftime('%Y-%m').tolist()

# Apply new index and datetime conversion
for RGI_region, region_name in regions.items():
    regional_solid_prcp[RGI_region].columns = new_indices
    regional_solid_prcp[RGI_region].columns = pd.to_datetime(new_indices)
    regional_liq_prcp[RGI_region].columns = new_indices
    regional_liq_prcp[RGI_region].columns = pd.to_datetime(new_indices)
    areas[RGI_region].columns = new_indices
    areas[RGI_region].columns = pd.to_datetime(new_indices)

In [42]:
# We only use initial area to compute total volume of precip components

total_liq_prcp = {} 
total_solid_prcp = {} 

for RGI_region, region_name in regions.items():
    total_liq_prcp[RGI_region] = pd.concat([areas[RGI_region] * regional_liq_prcp[RGI_region]], axis=1)
    total_solid_prcp[RGI_region] = pd.concat([areas[RGI_region] * regional_solid_prcp[RGI_region]], axis=1)

In [43]:
total_prcp = {} 

for RGI_region, region_name in regions.items():
    total_prcp[RGI_region] = total_solid_prcp[RGI_region] + total_liq_prcp[RGI_region]

#### Now aggregating by basin

In [44]:
import json
def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/rgi_ids_per_basin.json'  
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [45]:
def sum_basin(basin_RGI_list, precip_data):
    # Create new list to match our RGI formatting
    new_basin_list = [str(x)[-8:] for x in basin_RGI_list]
    
    # Filter new_basin_list to keep only the indexes present in the DataFrame
    new_basin_list = [x for x in new_basin_list if x in precip_data.index]
    
    # Extract glaciers contained in the list from original df and create a new df
    new_df = precip_data.loc[new_basin_list].copy()
    
    # Sum the values of the glaciers within the basin
    summed_basin_precip = new_df.sum()
    #print(summed_basin_runoff)
    
    return summed_basin_precip

In [46]:
basin_gls = {}
for basin, ID in basins.items():
    basin_gls[basin] = select_glaciers_json(ID)

In [47]:
basin_precip_totals = {}
for basin, ID in basins.items():
    basin_precip_totals[basin] = {}
    for RGI_region, region_name in regions.items():
        basin_precip_totals[basin][RGI_region] = sum_basin(basin_gls[basin], total_prcp[RGI_region])

In [48]:
#Combining basin sums for same basins that cross RGI region boundaries
total_prcp = {}
for b, basin in enumerate(basins):
    total_prcp[basin] = 0
    for r, region in enumerate(regions):
        total_prcp[basin] += basin_precip_totals[basin].get(region, 0)

In [49]:
avg_prcp = {}
for b, basin in enumerate(basins):
    avg_prcp[basin] = total_prcp[basin][-20:].mean()

In [50]:
precipitation_list = []

# Iterate over basins to extract the mean total precipitation
for basin in basins:
    # Extract the mean total precipitation for the current basin
    mean_total_precipitation = avg_prcp[basin]
    precipitation_list.append(mean_total_precipitation)

# Create a pandas Series with basin names as index and mean total precipitation as values
precipitation_series = pd.Series(precipitation_list, index=basins)

In [51]:
precipitation_series

RHINE          1.080036e+09
RHONE          2.562059e+09
PO             8.501258e+08
DANUBE         1.006785e+09
TITICACA       5.161067e+08
                   ...     
GANGES         1.856076e+10
BRAHMAPUTRA    2.583841e+10
OB             1.423324e+09
INDIGIRKA      2.294758e+08
YANGTZE        2.646151e+09
Length: 75, dtype: float64

In [52]:
# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Precipitation Analysis/'

fname = f"precip_avgs_Glo.csv"

# Define the full path of the output file
output_path = os.path.join(output_dir, fname)

# Save the DataFrame as CSV
precipitation_series.to_csv(output_path, header=True, index=True)